![人工智慧 - 自由團隊](https://raw.githubusercontent.com/chenkenanalytic/img/master/af/aifreeteam.png)


<center>Welcome to the course《Python: from business analytics to Artificial Intelligence》by AI . FREE Team.</center>
<br>
<center>歡迎大家來到 AI . FREE Team 《Python 從商業分析到人工智慧》的 AI 基礎教學 - Lesson 08 AI Dream Interpretation (3-08)。 </center>
<br>

<center>(Author: Chen Ken；Date of published: 2021//；AI . FREE Team Website: https://aifreeblog.herokuapp.com/)</center>

# 解夢文本生成 - Dream Interpretation Text Generation

莊周夢蝶，哲學家討論著現實與夢境的虛實真假，究竟夢境中的情節是現實，還是只是反映生活的片段記憶？

又或是對於未來的預測、生物本能的第六感？因此從古代人們透過《易經》，卜卦解析夢境與現實的關聯，

進而預測未來事件的吉凶！解夢是機率統計？還是真的有其不可思議的力量？

讓我們透過 AI 來學習周易解夢的邏輯，試看看 AI 是否也能有預測未來的能力吧！

在上一篇教學，我們認識了 NLP 領域中的 NLU(自然語言理解)，本實作教學將帶讀者認識 NLG (自然語言生成)。

## STEP 0. 下載資料集並解壓縮

※ 資料集爬蟲自網路解夢相關網站，並已初步清理訓練文本。

In [ ]:
### 資料上傳時間，依電腦網速而定 (共三個壓縮檔)
!git clone https://github.com/AI-FREE-Team/AI-Introduction.git

In [ ]:
### 解壓縮資料集，路徑請依據實際情況調整
!unzip "/content/AI-Introduction/clean_all.zip" -d "/content"

## STEP 1. 讀取資料集

In [ ]:
### 讀取爬蟲解夢文字檔
f = open('/content/clean_all.txt')
text = str()
for line in f:
    text += line

### 檢視資料集

In [ ]:
text[6023:6200]

In [ ]:
# 檢視文字檔內容
n = len(text)
w = len(set(text))
print(f"解夢資料共有 {n} 中文字")
print(f"其中有 {w} 個獨一無二的字")

## STEP 2. 文字資料集清理

於本解夢實作，原始資料在爬蟲階段已經完成清理，以下程式碼取自 lesson 07 資料及清理方式，提供需要的讀者參酌。

In [ ]:
# # 導入相關文字清理套件

# ### beautifulsoup 為常見的爬蟲使用套件，擅長解析網頁原始碼 (ref: https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
# from bs4 import BeautifulSoup

# ### re 套件為正規表達式套件 (ref: https://docs.python.org/3/library/re.html)
# import re

# ### nltk 為自然語言處理常用套件 (ref: https://www.nltk.org/)
# import nltk
# from nltk.corpus import stopwords # 導入停用詞模組
# nltk.download('stopwords')
# nltk.download('punkt')

In [ ]:
# # 自定義文本清理的函數
# def Preprocessing(text):

#   # 將全部字詞轉為小寫
#   text = text.lower()

#   # 移除網頁編碼相關內容，例如：html原始碼
#   text = BeautifulSoup(text).get_text()

#   # 移除特殊字元
#   text = re.sub("[^a-zA-Z]"," ", text)

#   # 使用 nltk 套件，將文句切割成字詞列表
#   wordList = nltk.word_tokenize(text)

#   # # 使用字詞列表，刪除停用詞 -- 不使用
#   # filtered = [w for w in wordList if w not in stopwords.words('english')]

#   return " ".join(wordList)

In [ ]:
# # 使用 for loop 清理文本資料 (執行約數分鐘)
# for i in range(len(text))):
#   text[i] = Preprocessing(text[i])

In [ ]:
# # 快速檢視資料集
# text

## STEP 3. 建立字詞索引 (Word Indexing)

在進行自然語言訓練任務時，考量到神經網路的運算仍是以數字進行計算，因此我們必須建立數字索引對應到字詞，方能使神經網路進行訓練。

以中文字建立數值索引為例，如：｛ 一： 0 , 丁： 1 , ... ... , 龜： 3978 , ... ｝

In [ ]:
# 使用 keras 內建套件，建立字詞索引 (word indexing)
import tensorflow as tf
from keras.preprocessing.text import Tokenizer

# 先建立空的索引
tokenizer = Tokenizer(num_words = w, char_level = True, filters = '')

# 讀取文本，找出所有單一字詞建立索引
tokenizer.fit_on_texts(text)

# 將原始文本轉換成索引形式 (儲存在 sequences)
sequences = tokenizer.texts_to_sequences([text])[0]

# 將索引對應的字詞找出，計算共有幾個獨立字詞
word_index = tokenizer.word_index
print('共有 ',  len(word_index), ' 獨立字詞')

In [ ]:
# 檢視字詞索引
word_index

## STEP 4. 訓練資料集前處理

在文本生成的模型中，會將文本接續下一個字作為預測目標，簡單來說，第一個字預測目標為第二個字，第二字預測目標維第三個字...以此列舉，因此我們直接定義每一筆訓練數據的長度，label data 則為文本位移一個位置的數據。

```
eg.
train_X = text[0:128]
train_label = text[1:129]
```



In [ ]:
# 定義訓練輸入資料
SEQ_LENGTH = 128  # 每一段訓練文字長度
BATCH_SIZE = 128 # 每次訓練資料筆數

In [ ]:
# 我們目前的文本為 sequences (list)
# 需轉換為 tensorflow 讀取資料的格式
characters = tf.data.Dataset.from_tensor_slices(sequences)

# 將以數值索引的解夢內容
# 分成多個長度為 SEQ_LENGTH 的文句
# 並將文句長度不滿 SEQ_LENGTH 的句子刪去
data = characters.batch(SEQ_LENGTH + 1, drop_remainder=True)

In [ ]:
# 解夢訓練文本 - 成對的訓練資料組數
steps_per_epoch = len(sequences) // SEQ_LENGTH

# 透過此函式將每一段訓練資料
# 拆成兩個部分，(1) 少了最後一個字 (2) 少了第一個字
# 以利訓練資料做訓練 (資料集2為Label用途 → 對應每一個資料集1的接續文字)
def build_seq_pairs(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

# 將每個解夢文本索引資料套用上列公式
# 作為 輸入／輸出 的訓練資料 (input / target)
# 再透過 shuffle 將數據隨機排序
# 再透過 BATCH_SIZE（128） 去定義完整的訓練資料集
training_data = data.map(build_seq_pairs).shuffle(steps_per_epoch).batch(BATCH_SIZE, drop_remainder=True)

## STEP 5. 建置 NLP 類神經模型

In [ ]:
# 定義超參數
EMBEDDING_DIM = 512
LSTM_UNITS = 1024

# 使用 keras 定義模型
model = tf.keras.Sequential()

# 使用 word embedding 技術
model.add(tf.keras.layers.Embedding(input_dim=w, output_dim=EMBEDDING_DIM,batch_input_shape=[BATCH_SIZE, None]))

# 使用 LSTM 模型
model.add(tf.keras.layers.LSTM(units=LSTM_UNITS, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))

# 使用 dense 類神經網路
model.add(tf.keras.layers.Dense(w))

#查看模型
model.summary()

## STEP 6. 模型訓練

In [ ]:
# 定義超參數 (學習率)
LEARNING_RATE = 0.001

# 定義模型預測結果跟正確結果之間的差異 (LOSS)
# 因為全連接層沒使用 activation func
# from_logits= True
def loss(y_true, y_pred):
    return tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

# 使用 Adam 優化器
# 使用上方定義的損失函數
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss=loss)

In [ ]:
# 定義訓練次數
EPOCHS = 25

# 開始訓練
history = model.fit(
    training_data, # 前面建置的資料集
    epochs=EPOCHS #訓練次數
)

## STEP 7. 訓練模型儲存

In [ ]:
# 儲存訓練完成的模型
model.save('dream_interpretation_model_01.h5')

## STEP 8. 解夢模型設定

初步設定解夢模型架構，並讀取訓練模型權重。

In [ ]:
# 跟訓練時一樣的超參數，只差在 BATCH_SIZE 為 1
EMBEDDING_DIM = 512
LSTM_UNITS = 1024
BATCH_SIZE = 1

# 定義生成的模型
infer_model = tf.keras.Sequential()

# 使用 word embedding 技術
infer_model.add(tf.keras.layers.Embedding(input_dim=w, output_dim=EMBEDDING_DIM, batch_input_shape=[BATCH_SIZE, None]))

# 使用 LSTM 模型
infer_model.add(tf.keras.layers.LSTM(units=LSTM_UNITS, return_sequences=True, stateful=True))

# 使用 dense 類神經網路
infer_model.add(tf.keras.layers.Dense(w))

#查看模型
infer_model.summary()

In [ ]:
# 載入已儲存模型之權重
infer_model.load_weights('dream_interpretation_model_01.h5')
infer_model.build(tf.TensorShape([1, None]))

## STEP 9. 開始解夢

使用 infer_model 進行解夢！

In [ ]:
# 您的夢境
text_generated = '夢見老虎'

# 生成字數 → 建議不要太長或太短，20-40字內效果較佳
nums_text = 30

In [ ]:
# 使用 for loop 建立文本生成器
for i in range(nums_text):

  # 將夢境文本轉換成索引
  dream = tokenizer.texts_to_sequences([text_generated])[0]

  # 增加 batch 維度丟入模型取得預測結果後，再降維，拿掉 batch 維度
  input = tf.expand_dims(dream, axis=0)
  predictions = infer_model(input)
  predictions = tf.squeeze(predictions, 0)

  # 取得這個時間點模型生成的中文字
  predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
  partial_texts = [tokenizer.index_word[predicted_id]]
  text_generated += partial_texts[0]

In [ ]:
# 成功生成 解夢文字檔 → text_generated
# 透過擷取解夢內容作呈現
# print(text_generated)

# 使用此程式碼，讓解夢文字更有結構化
print(text_generated.split('\n')[0].split('。')[0])

#結論與發現

透過簡單的 LSTM 模型進行 NLG 任務 (自然語言生成)，可以發現成效仍有限，

但已經可以發現自然語言模型已能大致上瞭解字與字之間的關聯性，

若讀者有興趣深入理解其開發邏輯與原理，歡迎參考[《零基礎成為 AI 解夢大師秘笈》](https://aifreeblog.herokuapp.com/postlist/AI_DreamMaster/)，

文章將從零引導讀者認識 Python 語法、爬蟲、Python Web 開發、AI基礎理論、NLP基礎概念，

到最後引導讀者透過 Django 開發出使用者操作聊天介面！

本系列教學從 AI 基礎開始，到電腦視覺以及自然語言領域，

快速概要介紹了各式主流的 AI 核心技術，希望你/妳對於深度學習已經有基礎的認識！

# 更深入的教學與專案實作

如果你/妳對於深入開發 AI 技術的專案有興趣，想嘗試更多、更有趣、更扎實的實務AI技術專案，

歡迎參考我們從0到1的AI技術課程：[《學習 AI 一把抓：點亮人工智慧技能樹》](https://hahow.in/cr/slashie-ai-free-team)！

# 課程文件

### AI Foundations 課程清單
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 00 Preface 課程前言</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 01 MP model</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 02 Logistic Regression</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 03 Neural Network</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 04 Deep Neural Network</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 05 Convolution Neural Network</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 06 Handwriting Traditional Chinese Characters Recognition</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 07 Sentiment Analysis</a>
- <a href="https://colab.research.google.com/github/AI-FREE-Team/Python-Basics/blob/master/documents/Lesson00%20Preface.ipynb">Lesson 08 AI Dream Interpretation</a> (We are here now! --本篇課程--)